<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [2]:
data = pd.read_csv('../../data/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [5]:
y = data["quality"]
X = data[[x for x in data if x != 'quality']]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.3, random_state = 17)
scaler = StandardScaler()
all_scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
X_scaled = all_scaler.fit_transform(X)

In [6]:
import numpy.random
np.random.seed(42)
test_set = [[np.random.normal(12, 3), np.random.randint(10), np.random.randint(100, 200)] for _ in range(15)]
test_set_df = pd.DataFrame(test_set, columns=["norm", "ten", "big"])
#print(np.mean(test_set_df["norm"].values), " ", np.mean(test_set_df["ten"].values), " ", np.mean(test_set_df["big"].values))
#print(test_set_df)
X_ts = test_set_df[[x for x in test_set_df if x != "big"]]
y_ts = test_set_df["big"]

#print(X_ts)

X_ts_train, X_ts_holdout, y_ts_train, y_ts_holdout = train_test_split(X_ts, y_ts, test_size = 0.2, random_state = 17)
scaler_ts = StandardScaler()
X_ts_train_scaled = scaler_ts.fit_transform(X_ts_train, y_ts_train)
X_ts_holdout_scaled = scaler_ts.transform(X_ts_holdout)

'''
print(X_ts_train)
print(X_ts_train_scaled)
print(X_ts_holdout)
print(X_ts_holdout_scaled)
'''
None

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [7]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [8]:
#mean_squared_error?
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [9]:
# LinearRegression?
linreg_coef = pd.DataFrame(
   {'feat': X.columns,
    'coef': linreg.coef_.tolist()})
linreg_coef.sort_values(by="coef")

,coef,feat
7,-0.665720,density
1,-0.192260,volatile acidity
2,-0.000183,citric acid
4,0.008127,chlorides
6,0.014304,total sulfur dioxide
5,0.042180,free sulfur dioxide
9,0.062053,sulphates
0,0.097822,fixed acidity
10,0.129533,alcohol
8,0.150036,pH


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [10]:
lasso1 = Lasso(alpha = 0.01,random_state = 17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [11]:
#lasso1.coef_?
lasso1_coef = pd.DataFrame(
    {'feat': X.columns,
     'coef': lasso1.coef_.tolist()})
lasso1_coef.sort_values(by="coef") # Ваш код здесь
#lasso1_coef

,coef,feat
7,-0.235492,density
1,-0.188479,volatile acidity
4,-0.002747,chlorides
0,-0.000000,fixed acidity
2,-0.000000,citric acid
6,-0.000000,total sulfur dioxide
9,0.029722,sulphates
5,0.043088,free sulfur dioxide
8,0.067277,pH
3,0.256363,residual sugar


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [12]:
# ?LassoCV

# alphas = np.logspace(-6, 2, 200)
# lasso_cv = LassoCV(alphas = alphas, cv = 5, random_state = 17)
# lasso_cv.fit(X_scaled, y)
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas = alphas, cv = 5, random_state = 17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [13]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [14]:
lasso_cv_coef = pd.DataFrame({'feat': X.columns,
              'coef': lasso_cv.coef_.flatten().tolist()})
# lasso_cv_coef.sort_values?
lasso_cv_coef.sort_values(by="coef")

,coef,feat
7,-0.648161,density
1,-0.192049,volatile acidity
2,-0.000000,citric acid
4,0.006933,chlorides
6,0.012969,total sulfur dioxide
5,0.042698,free sulfur dioxide
9,0.060939,sulphates
0,0.093295,fixed acidity
10,0.137115,alcohol
8,0.146549,pH


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [15]:
# print("Mean squared error (train): %.3f" % # Ваш код здесь
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [40]:
forest = RandomForestRegressor(random_state = 17)
#почему в правильном ответе мы учимся на X_train_scaled???
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [58]:
#cross_val_score?
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train)))
print("Mean squared error (cv): %.3f" % np.abs(np.mean(cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout)))

Mean squared error (train): 0.076
Mean squared error (cv): 0.462
Mean squared error (test): 0.420


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [42]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params)
locally_best_forest.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [67]:
good_forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state = 17)
good_forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [68]:
print("Mean squared error (cv): %.3f" % np.abs(np.mean(cross_val_score(good_forest, X_train, y_train, scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, good_forest.predict(X_holdout)))

Mean squared error (cv): 0.457
Mean squared error (test): 0.409


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [51]:
rf_importance = pd.DataFrame({
    'feat': X.columns,
    'feature_rating': good_forest.feature_importances_})
rf_importance.sort_values(by = "feature_rating", ascending = False)

,feat,feature_rating
10,alcohol,0.218873
1,volatile acidity,0.118438
5,free sulfur dioxide,0.110910
7,density,0.083760
8,pH,0.072066
4,chlorides,0.071820
6,total sulfur dioxide,0.069788
3,residual sugar,0.068386
9,sulphates,0.062671
2,citric acid,0.061970


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**